In [2]:
ruta='/Users/Leon/Documents/Data GH/Mineria de textos/corpusTASS-2020'

In [3]:
import pandas as pd
import os

# Cargar archivos
df_train = pd.read_csv(os.path.join(ruta, 'train.tsv'), sep='\t')
df_test = pd.read_csv(os.path.join(ruta, 'test.tsv'), sep='\t')
df_dev = pd.read_csv(os.path.join(ruta, 'dev.tsv'), sep='\t')



In [4]:

# Filtrar columnas y eliminar filas con valores nulos
df_train_filtered = df_train[['texto', 'etiqueta']].dropna()
df_dev_filtered = df_dev[['texto', 'etiqueta']].dropna()
df_test_filtered = df_test[['texto', 'etiqueta']].dropna()

In [5]:
import re

# Haremos la limpieza de los textos con expresiones regulares

def limpiar_texto(texto):
    texto = re.sub(r'@\w+', '', texto)  # eliminar menciones
    texto = re.sub(r'http\S+|www\S+', '', texto)  # eliminar URLs
    texto = re.sub(r'#\w+', '', texto)  # eliminar hashtags
    texto = re.sub(r'[^\w\sáéíóúüÁÉÍÓÚÜ]', '', texto)  # eliminar acentos y caracteres especiales
    texto = texto.lower()  # convertir a minúsculas
    texto = re.sub(r'\s+', ' ', texto).strip()  # eliminar espacios extra
    texto = texto.encode('ascii', 'ignore').decode('ascii')  # eliminar emojis y simbolos no ASCII
    return texto

In [6]:
df_train_filtered['texto'] = df_train_filtered['texto'].astype(str).apply(limpiar_texto)
df_dev_filtered['texto'] = df_dev_filtered['texto'].astype(str).apply(limpiar_texto)
df_test_filtered['texto'] = df_test_filtered['texto'].astype(str).apply(limpiar_texto)


In [7]:
from datasets import Dataset
#convertimos a formato dataset
dataset_train = Dataset.from_pandas(df_train_filtered.reset_index(drop=True))
dataset_dev = Dataset.from_pandas(df_dev_filtered.reset_index(drop=True))
dataset_test = Dataset.from_pandas(df_test_filtered.reset_index(drop=True))

/Users/Leon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/Leon/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import pipeline


# Usa un modelo en español compatible con análisis de sentimiento desde Hugging Face
classifier = pipeline("text-classification", model="finiteautomata/beto-sentiment-analysis")

# Aplica el modelo a los textos del corpus para predecir el sentimiento.
def clasificar_sentimiento(ejemplo):
    resultado = classifier(ejemplo['texto'])
    return {'prediccion_modelo': resultado[0]['label'].lower()}  # minúscula para facilitar comparación

# Probamos
dataset_train = dataset_train.map(clasificar_sentimiento)
#dataset_dev = dataset_dev.map(clasificar_sentimiento)
#dataset_test = dataset_test.map(clasificar_sentimiento)

# Mostramos
print(dataset_train[0])



Map:  36%|███▋      | 1745/4802 [01:05<01:58, 25.69 examples/s]